In [2]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow

In [3]:

# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Untitled',
 'img',
 'models',
 '27.ipynb',
 '30.ipynb',
 'Face.zip',
 'FaceAll',
 '31.ipynb']

In [4]:
# Face detection model
faceDetectionModel = path + "/models/res10_300x300_ssd_iter_140000_fp16.caffemodel"

# Mô tả kiến trúc
faceDetectionProto = path + "/models/deploy.prototxt"

# Mô hình mô tả khuôn mặt
faceDescriptor = path + "/models/openface.nn4.small2.v1.t7"

# Sử dụng OpenCV DNN đọc mô hìn nhận diện khuôn mặt
dectectorModel = cv2.dnn.readNetFromCaffe(faceDetectionProto, faceDetectionModel)

# Đọc mô hình mô tả khuôn mặt từ file Torch
descriptorModel = cv2.dnn.readNetFromTorch(faceDescriptor)

In [5]:
faces = path + "/FaceAll/FaceAll"
# Hiển thị tất cả thư mục hoặc file
os.listdir(faces)

['1721030500_TranMaiDucLuong',
 '1721030651_VuDinhLam',
 '1721021119_TranDuyHa',
 '1721030867_TranDongAnh',
 '123456789_LeNhatTung',
 '1721030693_TranMinhQuyet',
 '172100338_CaoQuocAnh',
 '172100150_nguyensyhien',
 '162001391_TranTien',
 '1721030497_HoThienSinh',
 '162000309_LeHoangQuan',
 '172100337_HaDuyPhuong',
 '1721030799_NguyenNhatHieu',
 '162000186_NguyenVanHuong',
 '172100264_NgoDangDuy',
 '1721030403_LeKhaHieu',
 '1721030548_VuNguyenQuocAn',
 '1721031139_BaoNgocThienBao',
 '1721030881_ThanTrongPhuc',
 '1721031620_PhanQuocHuy',
 '172100041_HoangHuuDien',
 '1721030936-phamtrantrunghieu',
 '1721030961_NguyenPhuocSang',
 '1721031618_TruongThanhLuan',
 '1721030572_DoTruongPhatDat',
 '1721030687_TrinhDuyNghia',
 '172100201_NguyenThanhDat',
 '1721030537-VoHuuThang',
 '1721031324_LeVanTrong',
 '1721030848_LeQuocKy',
 '1721030491_Nguyễn Thế Nhật',
 '162000670_nguyenmanhthi',
 '1721031276_ThachAnhQuoc',
 '1721031129_TranAnhHao',
 '172100257_LuongThanhPhong',
 '1721030846_MaiHoTien

[[ 6.99123964e-02  4.01703082e-02 -9.12359431e-02  6.42130822e-02
  -1.40533326e-02  1.36742160e-01  1.13879450e-01  4.60036509e-02
  -1.47460967e-01  7.10726157e-02  6.46496415e-02  5.02544120e-02
   1.72739223e-01 -1.22236744e-01 -2.96903159e-02 -4.02826890e-02
  -3.96717824e-02  8.18619654e-02 -1.03994921e-01  1.58840165e-01
   6.40023053e-02 -1.83366597e-01  5.96084855e-02  9.37908664e-02
  -4.62522022e-02 -1.38541386e-01 -1.48177639e-01 -1.74584523e-01
  -6.07038327e-02  8.18131343e-02 -3.63502055e-02  5.26006632e-02
  -8.50538164e-02  7.70578161e-02 -8.29441920e-02 -2.21585762e-02
  -1.20444581e-01  2.01709429e-03 -9.07240734e-02  1.43319787e-02
  -2.40564905e-02 -1.48789272e-01 -2.52353866e-02  9.69382077e-02
  -1.32739350e-01 -1.12274550e-01  1.00084171e-01  1.79370753e-02
  -1.03897519e-01  1.52778760e-01  1.33027613e-01 -1.21608943e-01
  -1.37845337e-01 -5.35452086e-03 -1.97309591e-02 -8.30783974e-05
  -6.55663013e-02  6.43450022e-02  1.10021204e-01  1.40648615e-02
  -3.41686

In [6]:
# Trích xuất đặc trưng cho một ảnh
luan = path + '/FaceAll/FaceAll/172100304_LiuGiaLuan/img_300.jpg'
img1 = cv2.imread(luan)

# Copy cái ảnh
image = img1.copy()

# Lấy chiều cao và chiều rộng của ảnh
h, w = image.shape[:2]

# Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt
imgBlob = cv2.dnn.blobFromImage(image, 1, (300,300), (104, 177, 123), swapRB=False, crop=False)

# Thiết lập đầu vào cho mô hình
dectectorModel.setInput(imgBlob)

# Thực hiện việc nhận diện khuôn mặt
detections = dectectorModel.forward()

# Kiểm tra xem có khuôn mặt nào hay không?
if (len(detections)>0):
  # Chọn khuôn mặt có độ tin cậy cao nhất (confidence) cao nhất
  i = np.argmax(detections[0, 0, :, 2]) # Giá trị thứ 2 trong 7 giá trị cuối cùng => confidence
  confidence = detections[0, 0, i, 2]

  # Kiểm tra độ tin cậy có lớn hơn 0.5
  if (confidence>0.5):
      # Tính toán hộp bao quanh khuôn mặt
      box = detections[0, 0, i, 3:7]* np.array([w, h, w, h])
      (startX, startY, endX, endY) =box.astype('int')

      # Trích xuất vùng ảnh chứa khuôn mặt ra
      roi = image[startY:endY, startX:endX]
      cv2_imshow(roi)

      # Chuẩn bị dữ liệu đầu vào cho mô hình trích xuất đặc trưng
      faceBlob = cv2.dnn.blobFromImage(roi, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)

      # Thiết lập đầu vào cho mô hình
      descriptorModel.setInput(faceBlob)

      # Thực hiện việc trích xuất đặc trưng
      vectors = descriptorModel.forward()

      # Print vectors
      print(vectors)

In [7]:
# Tạo ra một functions để áp dụng cho nhiều ảnh
def myDetect(image_path):
  # Trích xuất đặc trưng cho một ảnh
  img1 = cv2.imread(image_path)

  # Copy cái ảnh
  image = img1.copy()

  # Lấy chiều cao và chiều rộng của ảnh
  h, w = image.shape[:2]

  # Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt
  imgBlob = cv2.dnn.blobFromImage(image, 1, (300,300), (104, 177, 123), swapRB=False, crop=False)

  # Thiết lập đầu vào cho mô hình
  dectectorModel.setInput(imgBlob)

  # Thực hiện việc nhận diện khuôn mặt
  detections = dectectorModel.forward()

  # Kiểm tra xem có khuôn mặt nào hay không?
  if (len(detections)>0):
    # Chọn khuôn mặt có độ tin cậy cao nhất (confidence) cao nhất
    i = np.argmax(detections[0, 0, :, 2]) # Giá trị thứ 2 trong 7 giá trị cuối cùng => confidence
    confidence = detections[0, 0, i, 2]

    # Kiểm tra độ tin cậy có lớn hơn 0.5
    if (confidence>0.5):
        # Tính toán hộp bao quanh khuôn mặt
        box = detections[0, 0, i, 3:7]* np.array([w, h, w, h])
        (startX, startY, endX, endY) =box.astype('int')

        # Trích xuất vùng ảnh chứa khuôn mặt ra
        roi = image[startY:endY, startX:endX]
        # cv2_imshow(roi)

        # Chuẩn bị dữ liệu đầu vào cho mô hình trích xuất đặc trưng
        faceBlob = cv2.dnn.blobFromImage(roi, 1/255, (96,96), (0,0,0), swapRB=True, crop=True)

        # Thiết lập đầu vào cho mô hình
        descriptorModel.setInput(faceBlob)

        # Thực hiện việc trích xuất đặc trưng
        vectors = descriptorModel.forward()

        # Print vectors
        return vectors

In [8]:
# Apply for all images
# Khởi tạo một dictionary
data = dict(data=[], label=[])

myDir = '/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll'
folders = os.listdir(myDir)

# Duyệt qua từng thư mục
for folder in folders:
  path = myDir + "/"+folder
  print(path)

  files =  os.listdir(path)
  for fileName in files:
    try:
        # Gọi hàm trích xuất đặc trưng
        vector = myDetect(path+'/'+fileName)

        # Kiểm tra việc trích xuất đặc trưng thành công:
        if vector is not None:
          #Thêm vector đặc trưng vào trong từ điển với nhãn là tên của thư mục
          data['data'].append(vector)
          data['label'].append(folder)
          print('Trích xuất đặc trưng thành công')
    except:
      # Bỏ qua nếu gặp lỗi
      pass

/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll/1721030500_TranMaiDucLuong
/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll/1721030651_VuDinhLam
/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll/1721021119_TranDuyHa
/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll/1721030867_TranDongAnh
/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll/123456789_LeNhatTung
/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll/1721030693_TranMinhQuyet
/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll/172100338_CaoQuocAnh
/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll/172100150_nguyensyhien
/content/drive/MyDrive/Colab Notebooks/FaceAll/FaceAll/162001391_TranTien
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc t

In [9]:
data.keys()

dict_keys(['data', 'label'])

In [10]:
# Tạo một series từ dictionary
labelSeries = pd.Series(data['label'])
# Đếm số lần xuất hiện của mỗi nhãn
labelCounts = labelSeries.value_counts()
print(labelCounts)

1721030566_LeNguyenHoang     150
1721030848_LeQuocKy          135
1721031129_TranAnhHao        130
172100257_LuongThanhPhong    121
1721030367_LePhuongLinh      119
                            ... 
1721030439_TranThanhBinh      10
1721030497_HoThienSinh         9
172100103_tranquangthuan       8
1721030802_NguyenNhatTien      5
1721030456_Đào Ngọc Huy      2
Name: count, Length: 91, dtype: int64


In [11]:
# Save data -> file
print('/content/drive/MyDrive/Colab Notebooks')
fileName = '/content/drive/MyDrive/Colab Notebooks/data_face_features.pickle'
pickle.dump(data, open(fileName, mode='wb'))

/content/drive/MyDrive/Colab Notebooks
